In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from simpledbf import Dbf5
from sklearn.cluster import KMeans

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


# Load Earlier Made Houses File

In [4]:
progress = gpd.read_file(r"C:\Users\bruno\Documents\J3Q3-4 BEP\repositories\test.shp")
#progress = progress.rename(columns = {'Closest_st':'C_PTS','closest_su':'C_Super', 'to_close_1': 'D_C_Super', 'to_closest': 'D_C_PTS'}).drop('cluster_te', axis =1)
#progress = progress[['name','fclass', 'osm_id', 'cluster', 'long', 'latt', 'C_PTS', 'D_C_PTS', 'C_Super', 'D_C_Super', 'geometry']]
progress

,name,fclass,osm_id,cluster,long,latt,C_PTS,C_Super,C_DoctorsI,C_SchoolsI,C_SportID,D_C_PTS,D_C_Super,D_C_Doctor,D_C_School,D_C_Sport,Accessibil,geometry
0,None,building,267999488,0,4.259434,52.049581,652095,159,96,90,90,238.997482,1296.024755,1747.563098,1115.489010,1115.489010,0,POINT (4.25943 52.04958)
1,None,building,267996968,2649,4.261053,52.050188,652095,159,96,90,90,146.379395,1203.406668,1654.945010,1022.870923,1022.870923,0,POINT (4.26105 52.05019)
2,None,building,267997481,4284,4.260152,52.050155,652095,159,96,90,90,166.724627,1223.751900,1675.290243,1043.216155,1043.216155,0,POINT (4.26015 52.05015)
3,None,building,265070349,1,4.343525,52.054926,480300,14,33,89,89,348.614352,868.088193,1999.932840,2519.517774,2519.517774,5,POINT (4.34353 52.05493)
4,None,building,265670500,594,4.341636,52.052332,480300,14,33,89,89,155.825721,675.299562,1807.144210,2326.729144,2326.729144,5,POINT (4.34164 52.05233)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009,None,building,267843259,5384,4.282780,52.048521,15996,29,96,104,104,140.761093,675.200116,1320.843847,1513.360350,1513.360350,4,POINT (4.28278 52.04852)
6010,Boshoek,building,229070095,5395,4.310450,52.041714,16669,129,91,76,76,542.478848,958.790792,1270.657488,936.360909,936.360909,4,POINT (4.31045 52.04171)
6011,None,building,265197726,5876,4.308207,52.042528,16669,129,91,76,76,619.134787,1035.446731,1347.313427,1013.016848,1013.016848,4,POINT (4.30821 52.04253)
6012,None,building,267709178,5754,4.277444,52.078013,229179,194,59,115,115,104.716581,573.933202,791.926412,609.025846,609.025846,7,POINT (4.27744 52.07801)


# Load Stops OD-matrix

In [29]:
dbf = Dbf5(r"C:\Users\bruno\Bureau RMC Dropbox\2. RMC Intern\1. Bureau\3. Stagiairs\Bruno\BHermans\qgis\od-matrix-stops.dbf")
stops = dbf.to_dataframe()
#stops = stops.pivot(index = 'origin_id', columns = 'destinatio', values = 'total_cost')


# Load OD-matrices facilities

In [30]:
dbf = Dbf5(r"C:\Users\bruno\Bureau RMC Dropbox\2. RMC Intern\1. Bureau\3. Stagiairs\Bruno\BHermans\qgis\Facilities OD-matrices goed\OD-matrix-Doctors.dbf")
Doctors = dbf.to_dataframe()
Doctors = Doctors.pivot(index = 'origin_id', columns = 'destinatio', values = 'total_cost')
Doctors['C_Doctor'] = Doctors.idxmin(axis=1)

dbf1 = Dbf5(r"C:\Users\bruno\Bureau RMC Dropbox\2. RMC Intern\1. Bureau\3. Stagiairs\Bruno\BHermans\qgis\Facilities OD-matrices goed\OD-matrix-Schools.dbf")
Schools = dbf1.to_dataframe()
Schools = Schools.pivot(index = 'origin_id', columns = 'destinatio', values = 'total_cost')
Schools['C_School'] = Schools.idxmin(axis=1)


dbf2 = Dbf5(r"C:\Users\bruno\Bureau RMC Dropbox\2. RMC Intern\1. Bureau\3. Stagiairs\Bruno\BHermans\qgis\Facilities OD-matrices goed\OD-matrix-Sport_Centres.dbf")
Sport_Centres = dbf2.to_dataframe()
Sport_Centres = Sport_Centres.pivot(index = 'origin_id', columns = 'destinatio', values = 'total_cost')
Sport_Centres['C_Sport'] = Sport_Centres.idxmin(axis=1)


# Extract closest facility and distance per stop

In [31]:
Origins = Doctors.index.tolist()
Closeststops = Doctors['C_Doctor'].tolist()
Distance = []
for i in range(902):
    Origin = Origins[i]
    Destination = Closeststops[i]
    Distance.append(Doctors.loc[Origin, Destination])
Doctors1 = pd.DataFrame({'Origin': Origins, 'Destination': Closeststops, 'Distance_to_Doctor':Distance})

Origins = Schools.index.tolist()
Closeststops = Schools['C_School'].tolist()
Distance = []
for i in range(902):
    Origin = Origins[i]
    Destination = Closeststops[i]
    Distance.append(Schools.loc[Origin, Destination])
Schools1 = pd.DataFrame({'Origin': Origins, 'Destination': Closeststops, 'Distance_to_Schools':Distance})

Origins = Sport_Centres.index.tolist()
Closeststops = Sport_Centres['C_Sport'].tolist()
Distance = []
for i in range(902):
    Origin = Origins[i]
    Destination = Closeststops[i]
    Distance.append(Sport_Centres.loc[Origin, Destination])
Sport_Centres1 = pd.DataFrame({'Origin': Origins, 'Destination': Closeststops, 'Distance_to_Sport_Centres':Distance})

In [32]:
StopsDoctors = stops.merge(Doctors1, left_on = 'destinatio', right_on = 'Origin')
StopsDoctors['D_C_Doctor'] = StopsDoctors['Distance_to_Doctor'] + StopsDoctors['total_cost']
StopsDoctors = StopsDoctors.dropna()

StopsSchools = stops.merge(Schools1, left_on = 'destinatio', right_on = 'Origin')
StopsSchools['D_C_School'] = StopsSchools['Distance_to_Schools'] + StopsSchools['total_cost']
StopsSchools = StopsSchools.dropna()

StopsSport_Centres = stops.merge(Sport_Centres1, left_on = 'destinatio', right_on = 'Origin')
StopsSport_Centres['D_C_Sport'] = StopsSport_Centres['Distance_to_Sport_Centres'] + StopsSport_Centres['total_cost']
StopsSport_Centres = StopsSport_Centres.dropna()


In [33]:
L = StopsDoctors.origin_id.unique().tolist()
distance = []
for i in L:
    distance.append({'origin': str(i), 'D_C_Doctor1': StopsDoctors[StopsDoctors['origin_id'] == i].D_C_Doctor.min(),
                    'C_DoctorsID':
                    StopsDoctors[(StopsDoctors['origin_id'] == i) &
                   (StopsDoctors['D_C_Doctor'] == StopsDoctors[StopsDoctors['origin_id'] == i]
                   .D_C_Doctor.min())]['Destination'].to_list()[0]
                    })
ClosestStopsDoctors = pd.DataFrame(distance)


L = StopsSchools.origin_id.unique().tolist()
distance = []
for i in L:
    distance.append({'origin': str(i), 'D_C_School1': StopsSchools[StopsSchools['origin_id'] == i].D_C_School.min(),
                    'C_SchoolsID':
                    StopsSchools[(StopsDoctors['origin_id'] == i) &
                   (StopsSchools['D_C_School'] == StopsSchools[StopsSchools['origin_id'] == i]
                   .D_C_School.min())]['Destination'].to_list()[0]
                    })
ClosestStopsSchools = pd.DataFrame(distance)


L = StopsSport_Centres.origin_id.unique().tolist()
distance = []
for i in L:
    distance.append({'origin': str(i), 'D_C_Sport1': StopsSport_Centres[StopsSport_Centres['origin_id'] == i].D_C_Sport.min(),
                    'C_SportID':
                    StopsSport_Centres[(StopsSport_Centres['origin_id'] == i) &
                   (StopsSport_Centres['D_C_Sport'] == StopsSport_Centres[StopsSport_Centres['origin_id'] == i]
                   .D_C_Sport.min())]['Destination'].to_list()[0]
                    })
ClosestStopsSport_Centres = pd.DataFrame(distance)


In [49]:
DoctorsMerge = progress.merge(ClosestStopsDoctors, left_on = 'C_PTS', right_on = 'origin')
DoctorsMerge['D_C_Doctor'] = DoctorsMerge['D_C_Doctor1'] + DoctorsMerge['D_C_PTS']

SchoolsMerge = DoctorsMerge.merge(ClosestStopsSchools, left_on = 'C_PTS', right_on = 'origin')
SchoolsMerge['D_C_School'] = SchoolsMerge['D_C_School1'] + SchoolsMerge['D_C_PTS']

Sport_Merge = SchoolsMerge.merge(ClosestStopsSport_Centres, left_on = 'C_PTS', right_on = 'origin')
Sport_Merge['D_C_Sport'] = Sport_Merge['D_C_Sport1'] + Sport_Merge['D_C_PTS']
#Sport_Merge = Sport_Merge[['name', 'fclass', 'osm_id', 'cluster', 'long', 'latt', 'C_PTS', 'C_Super', 'C_DoctorsID', 'C_SchoolsID', 'C_SportID', 'D_C_PTS', 'D_C_Super', 'D_C_Doctor', 'D_C_School', 'D_C_Sport', 'geometry']]
Final = Sport_Merge
Final

,name,fclass,osm_id,cluster,long,latt,C_PTS,C_Super,C_DoctorsI,C_SchoolsI,...,geometry,origin_x,D_C_Doctor1,C_DoctorsID,origin_y,D_C_School1,C_SchoolsID,origin,D_C_Sport1,C_SportID_y
0,None,building,267999488,0,4.259434,52.049581,652095,159,96,90,...,POINT (4.25943 52.04958),652095,1508.565616,96,652095,876.491528,90,652095,993.026075,48
1,None,building,267996968,2649,4.261053,52.050188,652095,159,96,90,...,POINT (4.26105 52.05019),652095,1508.565616,96,652095,876.491528,90,652095,993.026075,48
2,None,building,267997481,4284,4.260152,52.050155,652095,159,96,90,...,POINT (4.26015 52.05015),652095,1508.565616,96,652095,876.491528,90,652095,993.026075,48
3,None,building,265070349,1,4.343525,52.054926,480300,14,33,89,...,POINT (4.34353 52.05493),480300,1651.318488,33,480300,2170.903422,89,480300,1796.027013,38
4,None,building,265670500,594,4.341636,52.052332,480300,14,33,89,...,POINT (4.34164 52.05233),480300,1651.318488,33,480300,2170.903422,89,480300,1796.027013,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009,None,building,267843259,5384,4.282780,52.048521,15996,29,96,104,...,POINT (4.28278 52.04852),15996,1180.082754,96,15996,1372.599257,104,15996,1077.367851,46
6010,Boshoek,building,229070095,5395,4.310450,52.041714,16669,129,91,76,...,POINT (4.31045 52.04171),16669,728.178640,91,16669,393.882061,76,16669,1118.843626,12
6011,None,building,265197726,5876,4.308207,52.042528,16669,129,91,76,...,POINT (4.30821 52.04253),16669,728.178640,91,16669,393.882061,76,16669,1118.843626,12
6012,None,building,267709178,5754,4.277444,52.078013,229179,194,59,115,...,POINT (4.27744 52.07801),229179,687.209831,59,229179,504.309265,115,229179,887.670402,77


In [50]:
Final.columns
Final = Final[['name', 'fclass', 'osm_id', 'cluster', 'long', 'latt','C_PTS','D_C_PTS','C_Super',  'D_C_Super','C_DoctorsID','D_C_Doctor1','C_SchoolsID', 'D_C_School1',  'C_SportID_y', 'D_C_Sport1', 'geometry']]
Final

,name,fclass,osm_id,cluster,long,latt,C_PTS,D_C_PTS,C_Super,D_C_Super,C_DoctorsID,D_C_Doctor1,C_SchoolsID,D_C_School1,C_SportID_y,D_C_Sport1,geometry
0,None,building,267999488,0,4.259434,52.049581,652095,238.997482,159,1296.024755,96,1508.565616,90,876.491528,48,993.026075,POINT (4.25943 52.04958)
1,None,building,267996968,2649,4.261053,52.050188,652095,146.379395,159,1203.406668,96,1508.565616,90,876.491528,48,993.026075,POINT (4.26105 52.05019)
2,None,building,267997481,4284,4.260152,52.050155,652095,166.724627,159,1223.751900,96,1508.565616,90,876.491528,48,993.026075,POINT (4.26015 52.05015)
3,None,building,265070349,1,4.343525,52.054926,480300,348.614352,14,868.088193,33,1651.318488,89,2170.903422,38,1796.027013,POINT (4.34353 52.05493)
4,None,building,265670500,594,4.341636,52.052332,480300,155.825721,14,675.299562,33,1651.318488,89,2170.903422,38,1796.027013,POINT (4.34164 52.05233)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009,None,building,267843259,5384,4.282780,52.048521,15996,140.761093,29,675.200116,96,1180.082754,104,1372.599257,46,1077.367851,POINT (4.28278 52.04852)
6010,Boshoek,building,229070095,5395,4.310450,52.041714,16669,542.478848,129,958.790792,91,728.178640,76,393.882061,12,1118.843626,POINT (4.31045 52.04171)
6011,None,building,265197726,5876,4.308207,52.042528,16669,619.134787,129,1035.446731,91,728.178640,76,393.882061,12,1118.843626,POINT (4.30821 52.04253)
6012,None,building,267709178,5754,4.277444,52.078013,229179,104.716581,194,573.933202,59,687.209831,115,504.309265,77,887.670402,POINT (4.27744 52.07801)


# Running The Kmeans Algorithm

In [51]:
X = Final[['D_C_PTS', 'D_C_Super', 'D_C_Doctor1', 'D_C_School1', 'D_C_Sport1']]
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
a = kmeans.labels_
Final['Accessibility'] = a
Final

C:\Users\bruno\.conda\envs\geopandas1\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,name,fclass,osm_id,cluster,long,latt,C_PTS,D_C_PTS,C_Super,D_C_Super,C_DoctorsID,D_C_Doctor1,C_SchoolsID,D_C_School1,C_SportID_y,D_C_Sport1,geometry,Accessibility
0,None,building,267999488,0,4.259434,52.049581,652095,238.997482,159,1296.024755,96,1508.565616,90,876.491528,48,993.026075,POINT (4.25943 52.04958),1
1,None,building,267996968,2649,4.261053,52.050188,652095,146.379395,159,1203.406668,96,1508.565616,90,876.491528,48,993.026075,POINT (4.26105 52.05019),1
2,None,building,267997481,4284,4.260152,52.050155,652095,166.724627,159,1223.751900,96,1508.565616,90,876.491528,48,993.026075,POINT (4.26015 52.05015),1
3,None,building,265070349,1,4.343525,52.054926,480300,348.614352,14,868.088193,33,1651.318488,89,2170.903422,38,1796.027013,POINT (4.34353 52.05493),6
4,None,building,265670500,594,4.341636,52.052332,480300,155.825721,14,675.299562,33,1651.318488,89,2170.903422,38,1796.027013,POINT (4.34164 52.05233),6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009,None,building,267843259,5384,4.282780,52.048521,15996,140.761093,29,675.200116,96,1180.082754,104,1372.599257,46,1077.367851,POINT (4.28278 52.04852),2
6010,Boshoek,building,229070095,5395,4.310450,52.041714,16669,542.478848,129,958.790792,91,728.178640,76,393.882061,12,1118.843626,POINT (4.31045 52.04171),2
6011,None,building,265197726,5876,4.308207,52.042528,16669,619.134787,129,1035.446731,91,728.178640,76,393.882061,12,1118.843626,POINT (4.30821 52.04253),2
6012,None,building,267709178,5754,4.277444,52.078013,229179,104.716581,194,573.933202,59,687.209831,115,504.309265,77,887.670402,POINT (4.27744 52.07801),2


In [53]:
df1 = pd.DataFrame(kmeans.cluster_centers_)
df1.columns = ['D_C_PTS', 'D_C_Super', 'D_C_Doctor1', 'D_C_School1', 'D_C_Sport1']
df1

,D_C_PTS,D_C_Super,D_C_Doctor1,D_C_School1,D_C_Sport1
0,1280.899445,2591.218489,2859.786904,2338.764178,1315.238611
1,369.141347,1244.607002,1897.447965,1052.245518,1369.556129
2,224.158279,796.029395,733.512714,784.531824,892.268344
3,374.048714,1004.115133,2804.390244,2759.971264,890.792478
4,1508.429201,2741.342910,1592.692526,1289.900113,1401.739031
5,10332.132311,11443.497106,1054.285987,1652.771397,2234.395338
6,274.665974,1079.338996,1735.245813,1863.718828,1476.652913
7,229.780221,955.216633,913.161432,925.652346,1907.554695
8,226.522685,919.376456,1099.935223,1422.009082,3220.442158
9,451.297287,1342.823893,933.640067,1083.550711,975.678522


In [52]:
Final.to_file('test-version2.shp')

In [77]:
df1.to_excel('Cluster.xlsx')